In [1]:
import multiprocessing
import time
import numpy as np
import scipy as sp
import emcee
import testkit
import ciamod
import os
import gc
import sys
import pickle
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# set up the model arguments the drop these into theta(state vector) or runargs

# set up pressure grids in bar cos its intuitive
logcoarsePress = np.arange(-4.0, 2.5, 0.53)
#logcoarsePress = np.arange(-4.0, 3.0, 0.5)
coarsePress = 1000.* pow(10,logcoarsePress)
logfinePress = np.arange(-4.0, 2.4, 0.1)
finePress = 1000.* pow(10,logfinePress)
# forward model wants pressure in mbar
press = finePress
nprof = coarsePress.size
print nprof
print finePress.size

13
64


In [3]:
w1 = 1.0
w2 = 10.0
npatches = 1
pcover = np.array([npatches],dtype='f')
pcover[:] = 1.0
do_clouds = np.array([npatches],dtype='i')
do_clouds[:] = 1
cloudnum = np.array([npatches],dtype='i')
cloudnum[:] = 99
cloudtype = np.array([npatches],dtype='i')
cloudtype[:] = 1

use_disort = 0 

# now the linelist
# Set up number of gases, and point at the lists. see gaslist.dat
ngas = 4
gasnum = np.asfortranarray(np.array([1,2,6,3],dtype='i'))
lists = ["../Linelists/H2O_xsecs.pic","../Linelists/ch4_xsecs.pic","../Linelists/nh3_xsecs.pic","../Linelists/h2s_xsecs.pic"]


In [4]:
# get the basic framework from water list
rawwavenum, inpress, inlinetemps, inlinelist = pickle.load( open('..//Linelists/H2O_xsecs.pic', "rb" ) )
inpress=1000.*inpress
wn1 = 10000./w2
wn2 = 10000. / w1
inwavenum = np.asfortranarray(rawwavenum[np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1)))],dtype='float64')
ntemps = inlinetemps.size
npress= finePress.size
nwave = inwavenum.size
r1 = np.amin(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))
r2 = np.amax(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))

# Here we are interpolating the linelist onto our fine pressure scale.
# pickles have linelist as 4th entry....
linelist = (np.ones([ngas,npress,ntemps,nwave],order='F')).astype('float64', order='F')
for gas in range (0,ngas):
    inlinelist= pickle.load( open(lists[gas], "rb" ) )[3]
    for i in range (0,ntemps):
        for j in range (r1,r2+1):
            pfit = interp1d(np.log10(inpress),np.log10(inlinelist[:,i,j]))
            linelist[gas,:,i,(j-r1)] = np.asfortranarray(pfit(np.log10(finePress)))

linelist[np.isnan(linelist)] = -50.0


In [5]:
# cloudparams is structured array with 5 entries
# each one has a patch*cloud entries
#cloudparams = np.ones(5)
# 5 entries in cloudparams for simple slab model are:
# 0) log10(number density)
# 1) top layer id (or pressure)
# 2) base ID (these are both in 61 layers)
# 3) rg
# 4) rsig
##cloudparams[0] = -20.
#cloudparams[1] = 10
#cloudparams[2] = 12
#cloudparams[3] = 1e-4
#cloudparams[4] = 1e-5
# hardwired gas and cloud IDs
#cloudnum = np.array([1],dtype='i')



In [6]:
# Get the cia bits
tmpcia, ciatemps = ciamod.read_cia("CIA_DS_aug_2015.dat",inwavenum)
cia = np.asfortranarray(np.empty((4,ciatemps.size,nwave)),dtype='float32')
cia[:,:,:] = tmpcia[:,:,:nwave] 
ciatemps = np.asfortranarray(ciatemps, dtype='float32')


In [7]:
# hardwired FWHM of data
fwhm = 0.005
#fixvmrs = -8.0

# get the observed spectrum
obspec = np.asfortranarray(np.loadtxt("caro_test_spec_1_10.dat",dtype='d',unpack='true'))

# NEXT LINE ADDS SYTEMATIC TO SPECTRUM FOR Log F retrieval
#obspec[1,:] = obspec[1,:] + 0.1*(min(obspec[2,10::3]**2))


runargs = pcover, cloudtype,do_clouds,gasnum,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec



In [8]:
nprof = 13
ndim =(nprof + (ngas) + 10)
theta = np.empty(ndim)
print ndim

27


In [19]:
theta[0] = 0.5 * np.random.randn() - 3.5 # H2O
theta[1] = 0.5 * np.random.randn() - 3.5 # CH4
theta[2] = 0.5* np.random.randn() - 3.5 # NH3
theta[3] = 0.5* np.random.randn() - 5.0 # H2S
theta[4] = (3*np.random.rand()) # -19. # log ref density
theta[5] = 100. * np.random.rand() #cloud top
theta[6] = 100. * np.random.rand() #cloud bottom
#  radii in um
theta[7] = np.random.rand()# + 10.# rg
theta[8] = 0.5 * np.random.randn() # rsig
theta[9] = np.random.rand() + 4.2  # logg
theta[10] =  0.95 + 0.1* np.random.rand()   # r2d2
theta[11] = np.random.randn() * 0.001 # dlam
theta[12] =  50. + np.random.randn() # gamma
theta[13] = np.log10((np.random.rand() * (max(obspec[2,:]**2)*(10. - 0.01))) + (0.01*min(obspec[2,10::3]**2)))
BTprof = np.loadtxt("BTtemp800_45_13.dat")
theta[14:] = (BTprof-200) + (100. * np.random.rand())


In [20]:
print theta

[ -4.53978978e+00  -3.96917458e+00  -3.34916193e+00  -5.06592362e+00
   3.35754150e-01   8.89971210e+01   9.26920980e+01   2.48316370e-01
   1.46426813e-01   4.97456101e+00   9.73550031e-01   1.88020967e-04
   5.04455852e+01   1.01043735e+01   2.19577028e+02   2.46788593e+02
   2.76583932e+02   3.00083169e+02   3.31541353e+02   3.55329891e+02
   3.93056181e+02   4.86336625e+02   7.28256219e+02   1.07392576e+03
   1.43576652e+03   2.44260922e+03   3.36992856e+03]


In [21]:
print testkit.lnprob(theta,*runargs) #* -2. / (obspec.shape[1] /3.)
#print testkit.lnprob(theta2,*runargs) * -2. / (obspec.shape[1] /3.)


[[  1.00000000e+01   9.99000999e+00   9.98003992e+00 ...,   1.00020004e+00
    1.00010001e+00   1.00000000e+00]
 [  2.68310036e+02   3.24770213e+02   3.59900459e+02 ...,   4.79349841e+05
    4.27770307e+05   4.01226264e+05]]
-144965.380147


In [12]:
print pickle.load(open("../Linelists/H2O_xsecs.pic","rb"))[2]

[   75.    85.   100.   120.   140.   160.   180.   200.   230.   260.
   300.   350.   400.   500.   650.   800.  1000.  1200.  1400.  1600.
  1800.  2000.  2300.  2600.  3000.  3500.  4000.]


In [ ]:
print theta[14:]


In [ ]:
print abs(44.87)